In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

# citizens_st_mobile_parquet remaking

In [ ]:
start_date = datetime.datetime.strptime('2023-07-06','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-04-21','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.parquet')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
for day_index in dates_pd.index:
    date = dates_pd.loc[day_index,['date']].values[0]
    date_key = dates_pd.loc[day_index,['date_key']].values[0]
    query_text = f"""
        INSERT INTO FUNCTION s3(
        'https://storage.yandexcloud.net/dwh-asgard/citizens_st_mobile_parquet_uuid/{date_key}',
        'parquet'
        )
        SETTINGS s3_truncate_on_insert = 1
        SELECT
            citizens_st_mobile.report_date AS report_date,
            citizens_st_mobile.citizen_id AS citizen_id,
            citizens_st_mobile.trial_available AS trial_available,
            citizens_st_mobile.state AS state,
            citizens_st_mobile_ch_2025_05_30.flat_uuid AS flat_uuid,
            citizens_st_mobile_ch_2025_05_30.address_uuid AS address_uuid
        FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
        LEFT JOIN
            (SELECT
                citizen_id,
                flat_uuid,
                address_uuid
            FROM db1.citizens_st_mobile_ch 
            WHERE report_date = '2025-05-30') AS citizens_st_mobile_ch_2025_05_30
            ON citizens_st_mobile_ch_2025_05_30.citizen_id = citizens_st_mobile.citizen_id
        WHERE report_date = '{date}'
        
    """
    ch.query_run(query_text)
    print(date)
# ch.query_run(query_text)


2025-04-21
2025-04-20
2025-04-19
2025-04-18
2025-04-17
2025-04-16
2025-04-15
2025-04-14
2025-04-13
2025-04-12
2025-04-11
2025-04-10
2025-04-09
2025-04-08
2025-04-07
2025-04-06
2025-04-05
2025-04-04
2025-04-03
2025-04-02
2025-04-01
2025-03-31
2025-03-30
2025-03-29
2025-03-28
2025-03-27
2025-03-26
2025-03-25
2025-03-24
2025-03-23
2025-03-22
2025-03-21
2025-03-20
2025-03-19
2025-03-18
2025-03-17
2025-03-16
2025-03-15
2025-03-14
2025-03-13
2025-03-12
2025-03-11
2025-03-10
2025-03-09
2025-03-08
2025-03-07
2025-03-06
2025-03-05
2025-03-04
2025-03-03
2025-03-02
2025-03-01
2025-02-28
2025-02-27
2025-02-26
2025-02-25
2025-02-24
2025-02-23
2025-02-22
2025-02-21
2025-02-20
2025-02-19
2025-02-18
2025-02-17
2025-02-16
2025-02-15
2025-02-14
2025-02-13
2025-02-12
2025-02-11
2025-02-10
2025-02-09
2025-02-08
2025-02-07
2025-02-06
2025-02-05
2025-02-04
2025-02-03
2025-02-02
2025-02-01
2025-01-31
2025-01-30
2025-01-29
2025-01-28
2025-01-27
2025-01-26
2025-01-25
2025-01-24
2025-01-23
2025-01-22
2025-01-21

## Обновление данных после 2025-05-30

In [ ]:
start_date = datetime.datetime.strptime('2025-05-30','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-04-21','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.parquet')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
for day_index in dates_pd.index:
    date = dates_pd.loc[day_index,['date']].values[0]
    date_key = dates_pd.loc[day_index,['date_key']].values[0]
    query_text = f"""
        INSERT INTO FUNCTION s3(
        'https://storage.yandexcloud.net/dwh-asgard/citizens_st_mobile_parquet/{date_key}',
        'parquet'
        )
        SETTINGS s3_truncate_on_insert = 1
        SELECT
            report_date,
            citizen_id,
            trial_available,
            state,
            flat_uuid,
            address_uuid
        FROM db1.citizens_st_mobile_ch
        WHERE report_date = '{date}'
        
    """
    ch.query_run(query_text)
    print(date)
# ch.query_run(query_text)


2025-06-22
2025-06-21
2025-06-20
2025-06-19
2025-06-18
2025-06-17
2025-06-16
2025-06-15
2025-06-14
2025-06-13
2025-06-12
2025-06-11
2025-06-10
2025-06-09
2025-06-08
2025-06-07
2025-06-06
2025-06-05
2025-06-04
2025-06-03
2025-06-02
2025-06-01
2025-05-31
2025-05-30


In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.citizens_st_mobile_parquet_mv
"""

ch.query_run(query_text)

,elapsed_ns,query_id
0,893036,3cf8bf5c-7ab4-4fc8-872b-e22d8701cfbb


citizens_st_mobile_parquet_mv

In [12]:
query_text = """
SELECT
    *
FROM db1.citizens_st_mobile_parquet
WHERE flat_uuid != ''
ORDER BY report_date
limit 10
"""

ch.query_run(query_text)

,report_date,citizen_id,trial_available,state,flat_uuid,address_uuid
0,2025-04-21,1591856,0,activated,ac085ec3-53b0-4e0a-922c-88ccaf97cfc4,abac2898-7e6c-401c-908f-a45462718b0f
1,2025-04-21,1591855,0,activated,ecd363af-62e0-469f-ba35-d5aa598ef245,07cf6a78-cde5-4747-a3ef-261b53ecc6e4
2,2025-04-21,1591840,0,activated,7e0ca5e3-0a83-49b8-9d3e-f55853dd96c6,92659ac4-b56e-4925-ac77-7e9f8ac8c813
3,2025-04-21,1591851,0,activated,52e1a33a-e945-4098-acf8-415fe6856135,851a55f9-bb8d-42ff-a13c-b7ee95641135
4,2025-04-21,1591850,0,activated,a37b2c05-503c-42ce-84d7-3ab407b31ae6,6af560e0-5211-4f7b-8b30-8561462a5c8d
5,2025-04-21,1591849,0,activated,dc85c031-bd90-48fe-8877-cc6b460af738,e2c8b424-3e2e-4b56-b574-0b42af9ff2c7
6,2025-04-21,1591852,0,activated,1e5713e1-19e6-45a1-ae96-96c6f4b75387,e1e2e1ee-e9b0-45cc-9f7f-c8ee85576715
7,2025-04-21,1591848,0,activated,d6001433-cd23-4e5b-8deb-15a3da287e7c,f9768171-46d9-4cfd-8433-627102a27f4b
8,2025-04-21,1591844,0,activated,4207eeba-b75b-4931-980e-2c83e3e8f760,cfbda7fa-a08d-4a32-b22a-c2760c90b7bb
9,2025-04-21,1591847,0,activated,9ad62be4-b2fd-4ff8-a866-364e1587caa8,5fe9193e-1358-41d4-ad8f-d649eccf0660


In [ ]:
query_text = """--sql
    CREATE TABLE db1.citizens_st_mobile_remake_parquet_ch
    (
    `report_date` Date,
    `citizen_id` Int32,
    `trial_available` Int32,
    `state` String,
    `flat_uuid` String,
    `address_uuid` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """

ch.query_run(query_text)

In [ ]:
query_text = """
CREATE MATERIALIZED VIEW db1.citizens_st_mobile_remake_parquet_mv
REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.citizens_st_mobile_remake_parquet_ch AS
INSERT INTO FUNCTION s3(
    'https://storage.yandexcloud.net/dwh-asgard/citizens_st_mobile_remake_parquet_ch/{date_key}',
    'parquet'
    )
    SETTINGS s3_truncate_on_insert = 1
    SELECT
        report_date,
        citizen_id,
        trial_available,
        state,
        flat_uuid,
        address_uuid
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    WHERE report_date = '{date}'
        
    """

ch.query_run(query_text)